In [37]:
# 모듈로딩
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
import glob
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [38]:
# 나이 선택 미성년층, 청년층, 장년층, 노년층을 대표할 수 있는 나이대를 선택
# 미성년층 : 10대, 청년층 : 20대, 장년층 : 40대, 노년층 : 60대
ages=[10,20,40,60]
file_path = './image/'

In [39]:
# ImageDataGenerator 인스턴스 생성
datagen = ImageDataGenerator(rescale=1./255,
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=[0.2, 2.2],
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

In [40]:
# image를 읽고, 리사이즈
# 초기 Grayscale로 인해, 정확도가 떨어짐. 그래서 주석 처리함.
dsize_ =(32,32)
imgdatas=[]
for age in ages:
    filename=file_path+str(age)+"/" +"*jpg*"
    fileList = glob.glob(filename)

    for file in fileList:
        # org =cv2.imread(file,cv2.IMREAD_GRAYSCALE)
        org =cv2.imread(file)
        img=cv2.resize(org,dsize_)
        imgdatas.append(img)

In [41]:
# 타겟 데이터 만들기
targetdatas = []
for age in ages:
    targetdata = []
    targetdata=[age]*20000
    targetdatas.extend(targetdata)
   
 # 원핫 인코딩
target=pd.get_dummies(targetdatas).values.astype(int)

In [42]:
# ImageDataGenerator 를 통해 다양한 각도 및 크기를 조절해서 학습
total = []
for x in range(len(imgdatas)):
    x = imgdatas[x].reshape((1,) + imgdatas[x].shape)
    j = 0
    for batch in datagen.flow(x=x, batch_size=1,shuffle=False):
        total.append(batch)
        j += 1
                
        if j > 199: # 각 원본 이미지당 새로운 이미지를 생성하는 개수 설정 
            break

In [43]:
total_numpy=np.array(total)

In [44]:
len(total_numpy)

80000

In [45]:
len(target)

80000

In [46]:
# train, test 분리
trainx,testx,trainy,testy = train_test_split(total_numpy,target,test_size=0.2,random_state=10,stratify=target)

In [47]:
trainx.shape

(64000, 1, 32, 32, 3)

In [48]:
trainx=trainx.reshape(64000, 32, 32, 3)

In [49]:
testx=testx.reshape(16000, 32, 32, 3)

In [50]:
trainy.shape

(64000, 4)

In [51]:
# 모듈 로딩
from keras.models import Sequential
from keras.layers import Conv2D,Dense,MaxPool2D,Flatten
from keras.callbacks import LearningRateScheduler,EarlyStopping,ModelCheckpoint
from keras.regularizers import L1,L2
import tensorflow as tf

In [52]:
# 전이 학습 사용
transfer_model=tf.keras.applications.VGG16(include_top=False,weights="imagenet",input_tensor=None,input_shape=(32,32,3))
transfer_model.trainable=False

In [53]:
# 모델링
model1 = Sequential()
model1.add(transfer_model)
model1.add(Conv2D(10,3,input_shape=(64,64,3),activation="relu",kernel_regularizer="l1",kernel_initializer="he_normal"))
model1.add(MaxPool2D())
model.add(Conv2D(10,3,input_shape=(64,64,3),activation="relu",kernel_regularizer="l1",kernel_initializer="he_normal"))
model.add(MaxPool2D())
model1.add(Flatten())
model1.add(Dense(100,activation="relu",kernel_regularizer="l2",kernel_initializer="he_normal"))
model1.add(Dense(100,activation="relu",kernel_regularizer="l2",kernel_initializer="he_normal"))
model1.add(Dense(100,activation="relu",kernel_regularizer="l2",kernel_initializer="he_normal"))
model1.add(Dense(4,activation="softmax"))

In [54]:
# 모델 컴파일, optimizer : adam, 손실함수 : categorical_crossentropy
model1.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [55]:
# callback mc,es,lr 설정

# model checkpoint
model_path = "./models/"
model_file = model_path+"best_model"
mc = ModelCheckpoint(model_file,save_best_only=True)

# Early Stopping
es = EarlyStopping(patience=5)


# Learing rate
def scheduler(epoch,lr):
    if epoch >20:
        return 0.0001
    else:
        return lr
lr = LearningRateScheduler(scheduler)

In [56]:
# 모델 학습
model1.fit(x=trainx,y=trainy,epochs=50,callbacks=[mc,es,lr],validation_split=0.2)

In [57]:
from keras.models import load_model
from keras.utils import custom_object_scope

In [60]:
# 커스텀 레이어 정의
class CastToFloat32(keras.layers.Layer):
    def call(self, inputs):
        return tf.cast(inputs, tf.float32)

# 모델 로드 시 custom_objects 옵션 사용
# with keras.utils.custom_object_scope({'CastToFloat32': CastToFloat32}) :
test3_model = load_model("./models/auto_model.h5",custom_objects=({ "Custom>CastToFloat32": CastToFloat32}))

In [62]:
test3_model.evaluate(testx,testy)

500/500 [==============================] - 7s 14ms/step - loss: 0.2477 - accuracy: 0.9103


[0.24768157303333282, 0.9102500081062317]

In [68]:
test3_model.get_build_config()

{'input_shape': (None, 32, 32, 3)}

In [69]:
test3_model.get_compile_config()

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adam',
  'config': {'name': 'Adam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': False,
   'is_legacy_optimizer': False,
   'learning_rate': 9.999999747378752e-05,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07,
   'amsgrad': False},
  'registered_name': None},
 'loss': {'classification_head_1': {'module': 'keras.losses',
   'class_name': 'CategoricalCrossentropy',
   'config': {'reduction': 'auto',
    'name': 'categorical_crossentropy',
    'from_logits': False,
    'label_smoothing': 0.0,
    'axis': -1,
    'fn': 'categorical_crossentropy'},
   'registered_name': None}},
 'metrics': [[{'module': 'keras.metrics',
    'class_name': 'MeanMetricWrapper',
    'config': {'name': 'accuracy',
     'dtype': 'float32',
     'fn': {'module': 'builtins',
      'class_name':

In [70]:
test3_model.get_config()

{'name': 'model',
 'trainable': True,
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 32, 32, 3),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_1'},
   'registered_name': None,
   'name': 'input_1',
   'inbound_nodes': []},
  {'module': None,
   'class_name': 'CastToFloat32',
   'config': {'name': 'cast_to_float32',
    'trainable': True,
    'dtype': 'float32'},
   'registered_name': 'CastToFloat32',
   'build_config': {'input_shape': (None, 32, 32, 3)},
   'name': 'cast_to_float32',
   'inbound_nodes': [[['input_1', 0, 0, {}]]]},
  {'module': 'keras.layers',
   'class_name': 'Normalization',
   'config': {'name': 'normalization',
    'trainable': True,
    'dtype': 'float32',
    'axis': (-1,),
    'invert': False,
    'mean': None,
    'variance': None},
   'registered_name': None,
   'build_config': {'input_shape': (None, 32, 32, 3)},
   'name': 'normalization',
   'inbound_nodes'